In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from Class_utils.parameters import file_paths,job_graph_par
from Class_utils.JobGraph import JobGraph
import ast
from Class_utils.FitnessFunctions import FitnessFunctions

%reload_ext autoreload
%autoreload 2

In [2]:
job_graph = JobGraph(file_paths, **job_graph_par)

Cache found loading...done


In [3]:
offer = pd.read_csv("../outputs/0_job_offers.csv").set_index("qId")

In [4]:
offer_md = pd.json_normalize(offer["metadata"].apply(ast.literal_eval))
offer_md.index.rename("qId", inplace=True)

In [5]:
es_comp_columns = ["Competence_essential"+str(i) for i in range(0, 3 + 1)]
op_comp_columns = ["Competence_optional"+str(i) for i in range(0, 2 + 1)]
es_know_columns = ["Knowledge_essential"+str(i) for i in range(0, 3 + 1)]
op_know_columns = ["Knowledge_optional"+str(i) for i in range(0, 2 + 1)]

In [6]:
counter = np.zeros((4,5))
qId_error_comp_ess,qId_error_comp_opt,qId_error_know_ess,qId_error_know_opt = [],[],[],[]


for qId, job in tqdm(offer.iterrows(),total=len(offer)):

    essential_comp = FitnessFunctions.filter(job[es_comp_columns].to_list())
    optional_comp = FitnessFunctions.filter(job[op_comp_columns].to_list())
    essential_know = FitnessFunctions.filter(job[es_know_columns].to_list())
    optional_know = FitnessFunctions.filter(job[op_know_columns].to_list())

    un_es_comp, amb_es_comp = job_graph.skill_standardize(essential_comp)
    un_op_comp, amb_op_comp = job_graph.skill_standardize(optional_comp)
    un_es_know, amb_es_know = job_graph.skill_standardize(essential_know)
    un_op_know, amb_op_know = job_graph.skill_standardize(optional_know)

    contex = un_es_comp + un_op_comp + un_es_know + un_op_know

    de_amb_es_comp = job_graph.solve_ambiguation(amb_es_comp, contex)
    de_amb_op_comp = job_graph.solve_ambiguation(amb_op_comp, contex)
    de_amb_es_know = job_graph.solve_ambiguation(amb_es_know, contex)
    de_amb_op_know = job_graph.solve_ambiguation(amb_op_know, contex)

 
    real_uris = offer_md.loc[qId, "uri_competence_essential"]
    equal_ = 0 if set(un_es_comp) | set(de_amb_es_comp)  ==  set(real_uris) else 1
    if equal_ == 1:
        counter[0, len(amb_es_comp)] += 1
        qId_error_comp_ess.append(qId)

    real_uris = offer_md.loc[qId, "uri_competence_optional"]
    equal_ = 0 if set(un_op_comp) | set(de_amb_op_comp)  ==  set(real_uris) else 1
    if equal_ == 1:
        counter[1, len(amb_op_comp)] += 1
        qId_error_comp_opt.append(qId)

    real_uris = offer_md.loc[qId, "uri_knowledge_essential"]
    equal_ = 0 if set(un_es_know) | set(de_amb_es_know)  ==  set(real_uris) else 1
    if equal_ == 1:
        counter[2, len(amb_es_know)] += 1
        qId_error_know_ess.append(qId)

    real_uris = offer_md.loc[qId, "uri_knowledge_optional"]
    equal_ = 0 if set(un_op_know) | set(de_amb_op_know)  ==  set(real_uris) else 1
    if equal_ == 1:
        counter[3, len(amb_op_know)] += 1
        qId_error_know_opt.append(qId)
    

100%|██████████| 220/220 [00:00<00:00, 778.28it/s]


In [7]:
counter

array([[0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 8., 3., 0., 0.],
       [0., 2., 0., 0., 0.]])

In [8]:
counter

array([[0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 8., 3., 0., 0.],
       [0., 2., 0., 0., 0.]])

In [9]:
1 - counter.sum(axis=1)/220

array([0.99090909, 1.        , 0.95      , 0.99090909])

In [10]:
# iiid = 171
# 
# essential_comp = FitnessFunctions.filter(offer.loc[iiid][es_comp_columns].to_list())
# print("Essential competence",essential_comp,"\n")
# optional_comp = FitnessFunctions.filter(offer.loc[iiid][op_comp_columns].to_list())
# print("Optional competence",optional_comp,"\n")
# essential_know = FitnessFunctions.filter(offer.loc[iiid][es_know_columns].to_list())
# print("Essential knowledge",essential_know,"\n")
# optional_know = FitnessFunctions.filter(offer.loc[iiid][op_know_columns].to_list())
# print("Optional knowledge", optional_know,"\n")
# 
# un_es_comp, amb_es_comp = job_graph.skill_standardize(essential_comp)
# un_op_comp, amb_op_comp = job_graph.skill_standardize(optional_comp)
# un_es_know, amb_es_know = job_graph.skill_standardize(essential_know)
# un_op_know, amb_op_know = job_graph.skill_standardize(optional_know)
# 
# contex = set(un_es_comp) | set(un_op_comp) | set(un_es_know) | set(un_op_know)
# 
# print("unique",un_es_comp,"\n")
# 
# print("ambigue",amb_es_comp,"\n")
# 
# deamb_es_comp = job_graph.solve_ambiguation(amb_es_comp, un_es_comp, True)
# # print("node similarity", job_graph.node_similarity(amb_op_comp[0], contex,True))
# 
# print("deambigued",deamb_es_comp,"\n")
# real_uri = offer_md.loc[iiid, "uri_competence_essential"]
# print("real",real_uri,"\n")
# print(set(un_es_comp) | set(deamb_es_comp) == set(real_uri))
# # essential_comp = FitnessFunctions.filter(job[es_comp_columns].to_list())
# # optional_comp = FitnessFunctions.filter(job[op_comp_columns].to_list())
# # essential_know = FitnessFunctions.filter(job[es_know_columns].to_list())
# # optional_know = FitnessFunctions.filter(job[op_know_columns].to_list())